# Import

In [1]:
import numpy as np
import random
from scipy.stats import spearmanr, pearsonr

#import the custom functions
import MFunctions as mf

In [2]:
#Load verb_symilarity which is the same for all
simdict = mf.verb_similarity("SimVerb-3500.txt")

## Define the vector dictionary

In [3]:
# Make observables object
set5 = mf.observables(["Linear.txt","Quadratic.txt","Cubic1.txt","Quartic1.txt","Additional1.txt"])
# Load matrices and make vector dictionaries
objsubset5 = mf.vector_dictionary("matrices_1160_arg_obj_context_subj.txt",set5)
subobjset5 = mf.vector_dictionary("matrices_1160_arg_subj_context_obj.txt",set5,[0])
obj08sub02set5 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.8,0.2]],set5,[1])
obj09sub01set5 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.9,0.1]],set5,[1])

## Length of hypernyms vs hyponyms

In [12]:
# Import the natural language processing libraries
#import nltk
from nltk.corpus import wordnet as wn

In [13]:
# Function to extract all the hyponyms of a given string
def get_hyponyms(synset):
    hyponyms = set()
    for hyponym in synset.hyponyms():
        hyponyms |= set(get_hyponyms(hyponym))
    return hyponyms | set(synset.hyponyms())

In [14]:
# Function to test whether the second word is a hyponym of the first
def is_hyponym(hyper,hypo):
    
    #Extract all possible meanings of the given hyper string
    allsyn = wn.synsets(hyper)
    
    # Initialise allhyponyms var to empty set
    allhypo = set()
    
    #Loop over all the possible meanings of hyper, thus allsyn, and search for all the associated hyponyms
    for syn in allsyn:
        allhypo |= get_hyponyms(syn)
    
    #Check if the given hypo has any overlap with the set of hyponyms found
    return allhypo.intersection(set(wn.synsets(hypo))) != set()

## Testing the lengths

In [69]:
pairlist = [list(a) for a in np.array(simdict['HYPER/HYPONYMS']).T[:2].T]

In [156]:
# Compare the lengths, how often is the hyper smaller than the hypo? Deviation vectors
compare = []
for a in pairlist[:243]+pairlist[250:515]+pairlist[550:800]:
    local = [mf.M_vec_mult_dev(objsubset5[a[0]].val,objsubset5[a[0]].val,objsubset5.mean,objsubset5.std),mf.M_vec_mult_dev(objsubset5[a[1]].val,objsubset5[a[1]].val,objsubset5.mean,objsubset5.std)]
    
    if is_hyponym(a[0],a[1]):
        compare.append(local[0]<local[1])
    else :
        compare.append(local[0]>local[1])
print(len(compare))
compare.count(False)/len(compare)

758


0.3733509234828496